# Aligning Llama 3 8B LLM to human preferences with Odds Ratio Preference Optimization (ORPO)

The challenge with RLHF or DPO is you need to keep aligning the SFT model with a reference model to make sure it doesn't stray away too far. This is where Odds Ratio Preference Optimization (ORPO) can be quite useful!

![](https://i.imgur.com/lQBBOkc.png)

ORPO aligns the language model without a reference model in a single-step manner by assigning a weak penalty to the rejected responses and a strong adaptation signal to the chosen responses with a simple log odds ratio term appended to the negative log-likelihood loss.

## What data is necessary for ORPO

For aligning an LLM to human preferences, you need triplets of preference data. Given a context/prompt, there is a preferred/good response which should be chosen over a dis-preferred/bad response. So each row of data should have a prompt, chosen and rejected response (typically created by humans). Same as DPO

## Preliminary setup for ORPO

![](https://i.imgur.com/6ZhLTNV.png)

## Objective Function of ORPO

![](https://i.imgur.com/fTiLYgv.png)

# Gradient of ORPO Loss

![](https://i.imgur.com/rgNgpbC.png)

In [1]:
import unsloth
import torch
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 1024

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A40. Max memory: 44.448 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [4]:
type(model)

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [5]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [6]:
# model = FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "Tell me about the capital of India?"},
]

prompt = tokenizer.apply_chat_template(messages,
                                       tokenize=False,
                                       add_generation_prompt=True)
print(prompt)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Tell me about the capital of India?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [7]:
# Encode the prompt.
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

# Generate the output.
output = model.generate(**inputs, max_new_tokens=200,
                        eos_token_id=tokenizer.eos_token_id,
                        tokenizer=tokenizer, stop_strings=["<|eot_id|>"])

# Decode the output.
text = tokenizer.decode(output[0], skip_special_tokens=False)

In [8]:
print(text)

<|begin_of_text|><|begin_of_text|><|start_header_id|>user<|end_header_id|>

Tell me about the capital of India?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The capital of India is New Delhi (officially known as the National Capital Territory of Delhi). It is a metropolitan city located in the northern part of the country, in the state of Delhi. New Delhi is the second-largest city in India by area and the third-largest by population, with a population of over 29.2 million people.

New Delhi is a planned city, built in the 20th century by the British, who wanted to create a new capital for the British Raj. The city was designed by the British architect Edwin Lutyens and the Viceroy of India, Lord Hardinge. The city was officially inaugurated as the capital of India in 1927.

New Delhi is known for its rich history, cultural heritage, and architectural landmarks. Some of the most famous landmarks in the city include:

1. The Red Fort (Lal Qila): A historic fort built by the M

In [9]:
import pandas as pd

dataset = load_dataset("Intel/orca_dpo_pairs", split="train")
dataset = dataset.shuffle(seed=42)
dataset = dataset.rename_column('question','prompt')
pd.set_option('display.max_colwidth', None)
df = dataset.to_pandas()
df.head()

system  \
0                                                                             You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.   
1  You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.   
2                                                                                                                                                                                             
3                                                                                          You are an AI assistant. You will be given a task. You must generate a detailed and long answer.   
4                                                                             You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              prompt  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [10]:
train_df = df.sample(frac=0.8, random_state=42)
val_df = df.drop(train_df.index)

train_df = train_df.sample(500, random_state=42)
val_df = val_df.sample(50, random_state=42)

In [11]:
train_df.head(2)

system  \
4458  You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.   
1609                                                                             You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              prompt  \
4458  Read the following context and choose the correct option to answer the question. Context: "That's Glenn Miller," my father said. "But it can't be." He had the back of the hospital bed cranked upright, the lower lid of his left eye creeping up in a warning signal I'd learned to recognize as a child. My older sister Ann had settled deep in the recliner, and she glared at me too, blaming me for winding him up. The jam box sat on the rolling tray table and my father was working the remote as he talked, backing up my newly burned CD and letting it spin forward to play a few seconds of low fidelity trombone solo. "You know the tune, of course," he said. "'King Porter Stomp.'" Those childhood years of listening to him play Glenn Miller on the console phonograph were finally paying off. "He muffed the notes the same way on the Victor version." "So why can't it be Miller?" I asked. "He wouldn't have played with a rabble like that." The backup musicians teetered on the edge of chaos, playing with an abandon somewhere between Dixieland and bebop. "They sound drunk." My father had a major emotional investment in Miller. He and my mother had danced to the Miller band at Glen Island Casino on Long Island Sound in the summer of 1942, when they were both sixteen. That signature sound of clarinet and four saxes was forever tied up for him with first love and the early, idea

In [12]:
def format_dpo_data_training(example):
    # Format system
    if len(example['system']) > 0:
        message = {"role": "system", "content": example['system']}
        system = tokenizer.apply_chat_template([message],
                                               tokenize=False)
    else:
        system = ""

    # Format instruction
    message = {"role": "user", "content": example['prompt']}
    prompt = tokenizer.apply_chat_template([message],
                                           tokenize=False,
                                           add_generation_prompt=True)

    # Format chosen answer
    chosen = example['chosen'] + "<|eot_id|>\n"

    # Format rejected answer
    rejected = example['rejected'] + "<|eot_id|>\n"

    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

In [13]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

original_columns = train_dataset.column_names

train_dataset = train_dataset.map(format_dpo_data_training,
                                  remove_columns=original_columns)
val_dataset = val_dataset.map(format_dpo_data_training,
                              remove_columns=original_columns)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [14]:
train_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 500
})

In [15]:
val_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 50
})

In [16]:
train_dataset[4]

{'prompt': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.<|eot_id|><|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nContinue writing the next sentence.\n\nWe see a couple of opening scenes. We see paint on a pallet. We see a person mixing oil paints. we<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
 'chosen': 'continue to see the artist carefully selecting and blending colors, as they meticulously work on their canvas, pouring their heart and emotions into each and every brushstroke.<|eot_id|>\n',
 'rejected': " Sure! I'd be happy to help you with your task. Based on the opening scenes you've described, it seems like we are looking at a scene of an artist mixing oil paints on a palette.\n\nStep 1: Identify the objects in the scene\n\n* Paint on a pallet\n* Pers

In [17]:
FastLanguageModel.for_training(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [18]:
peft_model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "up_proj", "down_proj", "gate_proj"],
    use_rslora=False,
    use_gradient_checkpointing="unsloth"
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.2.15 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [19]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
       

In [20]:
500 / 8

62.5

In [21]:
62 * 2

124

In [22]:
from trl import ORPOConfig

args = ORPOConfig(
        learning_rate=1.5e-5,
        lr_scheduler_type="linear",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        save_strategy="steps",
        eval_strategy="steps",
        # Set the logging steps.
        logging_steps=10,
        eval_steps=10, 
        save_steps=30,
        # Set the maximum number of training steps.
        max_steps=63,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=5,
        output_dir="output",
        max_prompt_length=512,
        max_length=max_seq_length,
        beta=0.1,
        seed=0,
    )

In [23]:
max_seq_length

1024

In [24]:
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [25]:
import trl
trl.__version__

'0.15.2'

In [26]:
from trl import ORPOTrainer

trainer=ORPOTrainer(
    model=peft_model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=args
)

/workspace/training-fine-tuning-large-language-models-workshop-dhs2024/Module-05-Aligning-Fine-tuned-LLMs-with-Human-Preferences-RLHF-PPO-DPO-ORPO/unsloth_compiled_cache/UnslothORPOTrainer.py:547: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map (num_proc=96):   0%|          | 0/500 [00:00<?, ? examples/s]

Map (num_proc=96):   0%|          | 0/500 [00:00<?, ? examples/s]

Map (num_proc=96):   0%|          | 0/500 [00:00<?, ? examples/s]

num_proc must be <= 50. Reducing num_proc to 50 for dataset of size 50.


...

num_proc must be <= 50. Reducing num_proc to 50 for dataset of size 50.


Map (num_proc=50):   0%|          | 0/50 [00:00<?, ? examples/s]

num_proc must be <= 50. Reducing num_proc to 50 for dataset of size 50.


Map (num_proc=50):   0%|          | 0/50 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [27]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 1
\        /    Total batch size = 8 | Total steps = 63
 "-____-"     Number of trainable parameters = 41,943,040


TrainOutput(global_step=63, training_loss=2.24258972349621, metrics={'train_runtime': 590.6941, 'train_samples_per_second': 0.853, 'train_steps_per_second': 0.107, 'total_flos': 0.0, 'train_loss': 2.24258972349621, 'epoch': 1.0})

In [31]:
# from getpass import getpass

# HF_TOKEN = getpass('Enter Huggingface Auth Token:')

Enter Huggingface Auth Token: ········


In [29]:
!rm -rf Llama3-8B-it-*

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
# peft_model.push_to_hub_merged("dipanjanS/Llama3-8B-it-orpo",
#                               tokenizer,
#                               save_method="merged_16bit",
#                               token='<token here>')

Unsloth: You are pushing to hub, but you passed your HF username = dipanjanS.
We shall truncate dipanjanS/Llama3-8B-it-orpo to Llama3-8B-it-orpo


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 325.05 out of 503.53 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 37.27it/s]


Unsloth: Saving tokenizer...

No files have been modified since last commit. Skipping to prevent empty commit.


...

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/dipanjanS/Llama3-8B-it-orpo


In [28]:
peft_model.save_pretrained_merged("Llama3-8B-it-orpo",
                                  tokenizer,
                                  save_method="merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 304.36 out of 503.53 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 35.64it/s]


Unsloth: Saving tokenizer... Done.
Done.


In [31]:
!ls -l --block-size=MB ./Llama3-8B-it-orpo

total 16078MB
-rw-rw-rw- 1 root root    1MB Feb 28 01:56 config.json
-rw-rw-rw- 1 root root    1MB Feb 28 01:56 generation_config.json
-rw-rw-rw- 1 root root 4977MB Feb 28 01:56 model-00001-of-00004.safetensors
-rw-rw-rw- 1 root root 5000MB Feb 28 01:56 model-00002-of-00004.safetensors
-rw-rw-rw- 1 root root 4916MB Feb 28 01:56 model-00003-of-00004.safetensors
-rw-rw-rw- 1 root root 1169MB Feb 28 01:56 model-00004-of-00004.safetensors
-rw-rw-rw- 1 root root    1MB Feb 28 01:56 model.safetensors.index.json
-rw-rw-rw- 1 root root    1MB Feb 28 01:56 special_tokens_map.json
-rw-rw-rw- 1 root root   18MB Feb 28 01:56 tokenizer.json
-rw-rw-rw- 1 root root    1MB Feb 28 01:56 tokenizer_config.json


In [32]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Llama3-8B-it-orpo", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = None,
        load_in_4bit = False,
    )

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A40. Max memory: 44.448 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [33]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
    

In [39]:
messages = [
    {"role": "system", "content": "You are an AI assistant that follows instruction extremely well. Help as much as you can."},
    {"role": "user", "content" : "Facts:   - sport: fencing    - death date: 6 june 1969      - birth date: 11 november 1886    - name: nicolaas nederpeld    - death place: the hague , netherlands    - birth place: the hague , netherlands   Based on these bullet points, write a short biography describing the life of nicolaas nederpeld .\nAnswer:"}
]

prompt = tokenizer.apply_chat_template(messages,
                                       tokenize=False,
                                       add_generation_prompt=True)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI assistant that follows instruction extremely well. Help as much as you can.<|eot_id|><|start_header_id|>user<|end_header_id|>

Facts:   - sport: fencing    - death date: 6 june 1969      - birth date: 11 november 1886    - name: nicolaas nederpeld    - death place: the hague , netherlands    - birth place: the hague , netherlands   Based on these bullet points, write a short biography describing the life of nicolaas nederpeld .
Answer:<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [40]:
# Encode the prompt.
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
# Generate the output.
output = model.generate(**inputs, max_new_tokens=256,
                        eos_token_id=tokenizer.eos_token_id,
                        tokenizer=tokenizer, stop_strings=["<|eot_id|>"])
# Decode the output.
text = tokenizer.decode(output[0], skip_special_tokens=False)
print(text)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI assistant that follows instruction extremely well. Help as much as you can.<|eot_id|><|start_header_id|>user<|end_header_id|>

Facts:   - sport: fencing    - death date: 6 june 1969      - birth date: 11 november 1886    - name: nicolaas nederpeld    - death place: the hague, netherlands    - birth place: the hague, netherlands   Based on these bullet points, write a short biography describing the life of nicolaas nederpeld.
Answer:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Nicolàas Nederpeld was a Dutch fencer who lived a remarkable life, spanning nearly 83 years. Born on November 11, 1886, in The Hague, Netherlands, he was destined to make a mark in the world of sports. As a talented fencer, he dedicated his life to the sport, honing his skills and competing at the highest levels.

Throughout his career, Nederpeld earned recognition for his exceptional abilities, but his life was

In [41]:
val_dataset[5]

{'prompt': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are an AI assistant that follows instruction extremely well. Help as much as you can.<|eot_id|><|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nFacts:   - sport: fencing    - death date: 6 june 1969      - birth date: 11 november 1886    - name: nicolaas nederpeld    - death place: the hague , netherlands    - birth place: the hague , netherlands   Based on these bullet points, write a short biography describing the life of nicolaas nederpeld .\nAnswer:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
 'chosen': 'Nicolaas Nederpeld was born on November 11, 1886, in The Hague, Netherlands. Growing up in his hometown, Nicolaas developed a strong passion for the sport of fencing. Over the years, his dedication and hard work paid off as he became known for his exceptional skills and prowess in the sport.\n\nAs he continued to practice and compete throughout his life, Nederpeld gained reco

In [42]:
subset_df = val_df.tail(20).copy()

In [44]:
from tqdm import tqdm

for idx, row in tqdm(subset_df.iterrows()):
    prompt = format_dpo_data_training(row)['prompt']
    print(prompt)
    break

0it [00:00, ?it/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI assistant. You will be given a task. You must generate a detailed and long answer.<|eot_id|><|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write the answer: The songs People Will Say We're in Love and The Surrey with the Fringe on Top are from which stage show?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [45]:
from tqdm import tqdm

responses = []
for idx, row in tqdm(subset_df.iterrows()):
    prompt = format_dpo_data_training(row)['prompt']
    # Encode the prompt.
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    # Generate the output.
    output = model.generate(**inputs, max_new_tokens=256,
                            eos_token_id=tokenizer.eos_token_id,
                            tokenizer=tokenizer, stop_strings=["<|eot_id|>"])
    # Decode the output.
    text = tokenizer.decode(output[0], skip_special_tokens=False)
    response = (text.split('<|start_header_id|>assistant<|end_header_id|>')[1]
                    .split('<|eot_id|>')[0]
                    .strip('\n'))
    responses.append(response)

20it [01:29,  4.50s/it]


In [46]:
subset_df.insert(2, 'ai_response', responses)

In [48]:
subset_df.head(20)

system  \
11966                                                                                          You are an AI assistant. You will be given a task. You must generate a detailed and long answer.   
6907                                                                                           You are an AI assistant. You will be given a task. You must generate a detailed and long answer.   
224                                                                                                                                 You are an AI assistant that helps people find information.   
8004   You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.   
12059                                                                                                                           Explain how you used the definition to come up with the answer.   
10578                                                                                          You are an AI assistant. You will be given a task. You must generate a detailed and long answer.   
11367                                                                                          You are an AI assistant. You will be given a task. You must generate a detailed and long answer.   
11149                                                                         You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.   
6082                                                                                                  You are an AI assistant that follows instruction extremely well. Help as much as you can.   
4569                                       You are an AI assistant that helps people find information. Provide a detailed answer so user don’t need to search outside to understand the answer.   
2740                                                                              You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.   
9949   You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.   
4555                                                                                           You are an AI assistant. You will be given a task. You must generate a detailed and long answer.   
5704                                                                                                                                                                                              
928    You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.   
9865   You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.   
9472                                                                              You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.   
5913                                                                                           You are an AI assistant. You will be given a task. You must generate a detailed and long answer.   
10429                                                                             You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.   
6928                                                                                           You are an AI assistant. You will be given a task. You must generate a detailed and long answer.   

                                                                                         

In [49]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key:  ········


In [50]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

In [51]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [52]:
from langchain_openai import OpenAIEmbeddings

embedder = OpenAIEmbeddings(model="text-embedding-3-large")

In [53]:
eval_df = subset_df[['ai_response', 'chosen']]
eval_df.columns = ['answer', 'ground_truth']
eval_df.insert(0, 'question', subset_df['system'].astype(str) + '\n\n' + subset_df['prompt'])

In [54]:
eval_df.head(2)

question  \
11966                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          You are an AI assistant. You will be given a task. You must generate a detailed and long answer.\n\nWrite the answer: The songs People Will Say We're in Love and The Surrey with the Fringe on Top are from which stage show?   
6907   You are an AI assistant. You will be given a task. You must generate a detailed and long answer.\n\nQ:Read the article and select the best answer. Article: The next generation of smart phone could combine the date from its gyroscopes   with a built-in compass to allow you to track your indoor movements even without GPS.          Research described it in the international Journal of Innovative Computing and Applications by Shahid Ayub of Lancaster University, and his colleagues there and at HW Communications, suggests that the embedded inertial sensors   in many smart phones have added the facility to be used for localization and tracking applications. The primary benefit of using smart phones is that no additional infrastructure would need to be installed for monitoring personnel movements in a wide variety of situations not least staff, equipment and inventory movements in warehouse facilities, shopping malls and factories. In contrast, other technology being proposed would require new equipment,such as RFID technologies. However, smart phones do not yet have the ability to pinpoint their location without recourse to the GPS system, which is not accessible indoors and has the added disadvantage of using up battery power very quickly even if it could be used. Inertial navigation only provides the necessary information   to a specified starting point the team says. This is useful for anyone with a non-GPS smart phone who wishes to track the route they take when walking or jogging if they spec

In [55]:
from ragas.metrics import answer_similarity, answer_correctness
from ragas import evaluate
from datasets import Dataset

eval_dataset = Dataset.from_pandas(eval_df)

results = evaluate(dataset=eval_dataset, metrics=[answer_correctness, 
                                     answer_similarity],
                   llm=chatgpt, embeddings=embedder)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

In [56]:
results

{'answer_correctness': 0.4994, 'semantic_similarity': 0.7485}

In [57]:
results.to_pandas().head(5)

user_input  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          You are an AI assistant. You will be given a task. You must generate a detailed and long answer.\n\nWrite the answer: The songs People Will Say We're in Love and The Surrey with the Fringe on Top are from which stage show?   
1  You are an AI assistant. You will be given a task. You must generate a detailed and long answer.\n\nQ:Read the article and select the best answer. Article: The next generation of smart phone could combine the date from its gyroscopes   with a built-in compass to allow you to track your indoor movements even without GPS.          Research described it in the international Journal of Innovative Computing and Applications by Shahid Ayub of Lancaster University, and his colleagues there and at HW Communications, suggests that the embedded inertial sensors   in many smart phones have added the facility to be used for localization and tracking applications. The primary benefit of using smart phones is that no additional infrastructure would need to be installed for monitoring personnel movements in a wide variety of situations not least staff, equipment and inventory movements in warehouse facilities, shopping malls and factories. In contrast, other technology being proposed would require new equipment,such as RFID technologies. However, smart phones do not yet have the ability to pinpoint their location without recourse to the GPS system, which is not accessible indoors and has the added disadvantage of using up battery power very quickly even if it could be used. Inertial navigation only provides the necessary information   to a specified starting point the team says. This is useful for anyone with a non-GPS smart phone who wishes to track the route they take when walking or jogging if they specify th

In [60]:
subset_df.insert(2, 'question', subset_df['system'].astype(str) + '\n\n' + subset_df['prompt'])
subset_df.head(1)

system  \
11966  You are an AI assistant. You will be given a task. You must generate a detailed and long answer.   

                                                                                                                           prompt  \
11966  Write the answer: The songs People Will Say We're in Love and The Surrey with the Fringe on Top are from which stage show?   

                                                                                                                                                                                                                             question  \
11966  You are an AI assistant. You will be given a task. You must generate a detailed and long answer.\n\nWrite the answer: The songs People Will Say We're in Love and The Surrey with the Fringe on Top are from which stage show?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          ai_response  \
11966  The two songs "People Will Say We're in Love" and "The Surrey with the Fringe on Top" are from the classic Rodgers and Hammerstein musical "Oklahoma!" The show premiered on March 31, 1943, at the St. James Theatre in New York City and was a groundbreaking production that revolutionized the genre of American musical theatre. The show was based on the 1939 play "Green Grow the Lilacs" by Lynn Riggs, which was set in the early 20th century in the American West.\n\nThe two songs in question are sung by the main characters, Laurey Williams and Curly McLain, as they develop their romantic relationship throughout the show. "People Will Say We're in Love" is a duet that captures the innocence and optimism of their blossoming romance, while "The Surrey with the Fringe on Top" is a charming and witty song in which Curly woos Laurey by describing the romantic and idyllic scene of a couple in love, set against the backdrop of a picturesque Surrey with a fringe on top.\n\nThe show's music was written by Richard Rodgers, with lyrics by Oscar Hammerstein II, and it features a range of memorable songs that have become iconic in the world of musical   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [61]:
import json

responses = []
for idx, row in tqdm(subset_df.head(2).iterrows()):
    prompt = f"""
                Act as an expert grader of responses from an AI model. 
                Follow these rules.
                  - Given the following question and AI answer, compare it to reference answers A1 and A2
                  - Return the response as a JSON with keys A1 and A2 and values of how close is the AI answer to each of them
                  - Represent closeness in a scale between 0 - 100 which is comparative
                  - Comparative closeness means sum of AI answer closeness to A1 and A2 should sum up to 100
                  - Example A1: 90, A2: 10 means comparing AI answer to A1 and A2, the AI answer is 90% closer to A1 and only 10% closer to A2
                  - Closeness should be measured by comparing how close is AI answer to A1 and A2 in terms of their meaning and context.
                  
                Return the response as a valid JSON dict only and not markdown JSON. 
                Do NOT return markdown.
                
                Question:
                {row['question']}
                
                AI Answer:
                {row['ai_response']}
                
                A1:
                {row['chosen']}
                
                A2:
                {row['rejected']}
            """
    response = chatgpt.invoke(prompt)
    response = response.content
    response = json.loads(response)
    responses.append(response)

print(responses)

2it [00:02,  1.41s/it]

[{'A1': 100, 'A2': 0}, {'A1': 95, 'A2': 5}]


In [65]:
responses = []
for idx, row in tqdm(subset_df.iterrows()):
    prompt = f"""
                Act as an expert grader of responses from an AI model. 
                Follow these rules.
                  - Given the following question and AI answer, compare it to reference answers A1 and A2
                  - Return the response as a JSON 
                  - Response should have keys A1, A2, Winner and values of how close is the AI answer to each of them and which answer is the winner
                  - Represent closeness in a scale between 0 - 100 which is comparative
                  - Comparative closeness means sum of AI answer closeness to A1 and A2 should sum up to 100
                  - Example A1: 90, A2: 10 means comparing AI answer to A1 and A2, the AI answer is 90% closer to A1 and only 10% closer to A2
                  - Closeness should be measured by comparing how close is AI answer to A1 and A2 in terms of their meaning and context.
                  
                Return the response as a valid JSON dict only and not markdown JSON.
                Do NOT return markdown.
                
                Question:
                {row['question']}
                
                AI Answer:
                {row['ai_response']}
                
                A1:
                {row['chosen']}
                
                A2:
                {row['rejected']}
            """
 
    response = chatgpt.invoke(prompt)
    response = response.content
    response = json.loads(response)
    responses.append(response)

20it [00:25,  1.28s/it]


In [66]:
response_df = pd.DataFrame(responses)
response_df

A1  A2 Winner
0   95   5     A1
1   90  10     A1
2   60  40     A1
3   70  30     A1
4   70  30     A1
5   80  20     A1
6   80  20     A1
7   30  70     A2
8   50  50    Tie
9   40  60     A2
10  80  20     A1
11  80  20     A1
12  70  30     A1
13  50  50    Tie
14  40  60     A2
15  60  40     A1
16  60  40     A1
17  60  40     A1
18  80  20     A1
19  50  50    Tie

In [67]:
response_df['Winner'].value_counts()

Winner
A1     14
A2      3
Tie     3
Name: count, dtype: int64